**Importing the libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import nltk
import scipy
import warnings
import re
warnings.filterwarnings('ignore')

**Reading the text files and observing its features**

In [2]:
# For train
df_train = pd.read_csv('labeledTrainData.tsv',sep='\t')

# For test
df_test = pd.read_csv('testData.tsv',sep='\t')


In [3]:
df_train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
df_test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
df_train.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


**Data Cleaning**

In [7]:
# Lets check review section
pd.options.display.max_colwidth = 500 # to view each rows text to 500 characters
print(df_train['review'])

0        With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle m...
1        \The Classic War of the Worlds\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for 

In [8]:
# Removing html tags from text.
# https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string

txt = " the source.Here's a pretensions:Just when"
cleanr = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',txt)
cleanr

" the source.Here's a pretensions:Just when"

In [9]:
# Removing digits from text
# https://stackoverflow.com/questions/817122/delete-digits-in-python-regex

text = " the source.Here's a 20 preten 103s and 120th sions:Just when"
text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Removing digits
text


" the source.Here's a preten 103s and 120th sions:Just when"

In [10]:
# Let's clean text
def clean_text(text):
  text = text.lower()
  text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',text)  # for removal of html tags
  text = re.sub(r"can't","cannot",text)
  text = re.sub(r"shan't","shall not",text)
  text = re.sub(r"won't","will not",text)
  text = re.sub(r"n't"," not",text) # see the space before not.
  text = re.sub(r"i'm","i am",text)
  text = re.sub(r"what's","what is",text)
  text = re.sub(r"let's","let us",text)
  text = re.sub(r"'re"," are",text)
  text = re.sub(r"'s"," ",text)  # space because we dont know the tense , it can be is/has anything.
  text = re.sub(r"'ve"," have",text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"\'scuse", " excuse ", text)
  text = re.sub(r"[^a-zA-Z]"," ",text)
  text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Removing digits
  text = re.sub('\W', ' ', text)  # If the comment/word does not contain any alphabets
  text = re.sub('\s+', ' ', text) # If there are more than one whitespace simultenously, then replace them by only 1 whitespace and also replace the punctuation marks
  text = text.strip(' ') # Removing leading and trailing white spaces
  return text


In [11]:
df_train['review'] = df_train['review'].apply(lambda text: clean_text(text))

In [12]:
print(df_train['review'])

0        with all this stuff going down at the moment with mj i have started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messa...
1        the classic war of the worlds by timothy hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate h g wells classic book mr hines succeeds in doing so i and those who watched his film with me appreciated the fact that it was not the standard predictable hollywood fare that comes out every year e g the spielberg version with tom cruise that had only the slightest resemblance to the book obviously everyone looks for different things

In [13]:
df_test['review'] = df_test['review'].apply(lambda text: clean_text(text))

In [14]:
print(df_test['review'])

0        naturally in a film who main themes are of mortality nostalgia and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones however there is a craftsmanship and completeness to the film which anyone can enjoy the pace is steady and constant the characters full and engaging the relationships and interactions natural showing that you do not need floods of tears to show emotion screams to show fear shouting to show dispute or violence to sho...
1        this movie is a disaster within a disaster film it is full of great action scenes which are only meaningful if you throw away all sense of reality let us see word to the wise lava burns you steam burns you you cannot stand next to lava diverting a minor lava flow is difficult let alone a significant one scares me to think that some might actually believe what they saw in this movie even worse is the significant amount of talent that went into making this film i mean the acting 

In [15]:
# For train file

X = df_train['review']
y = df_train['sentiment']

# For test file

X_test = df_test['review']

**Tokenizing text words**

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,roc_curve

In [17]:
tf = TfidfVectorizer(stop_words='english',max_features=20000)
tf

TfidfVectorizer(max_features=20000, stop_words='english')

In [18]:
# For training

X = tf.fit_transform(X)

# For test

X_test = tf.transform(X_test)

In [19]:
len(tf.get_feature_names_out()) ,
#tf.get_feature_names_out()

(20000,)

In [20]:
print(X.toarray().shape)
X.toarray()

(25000, 20000)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1,random_state=100)

**Building the model**

Using random forest classifier.

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()

In [23]:
rf_clf.fit(X_train,y_train)

RandomForestClassifier()

In [24]:
y_val_pred = rf_clf.predict(X_val)

In [25]:
print(accuracy_score(y_val_pred,y_val))

0.85


In [26]:
print(roc_curve(y_val_pred,y_val))

(array([0.       , 0.1561249, 1.       ]), array([0.        , 0.85611511, 1.        ]), array([inf,  1.,  0.]))


In [27]:
print(roc_auc_score(y_val_pred,y_val))

0.8499951039968665


In [28]:
rfc1 = rf_clf.predict(X_test)
rfc2 = rf_clf.predict_proba(X_test)[:,1]

In [29]:
rfc1

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [30]:
rfc2

array([0.87, 0.19, 0.66, ..., 0.57, 0.76, 0.55])

In [31]:
y_pred = rf_clf.predict(X_test)

In [32]:
y_pred

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [33]:
# putting result in submission file

submission_file = pd.read_csv('sampleSubmission.csv')
submission_file.head()

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0


In [34]:
submission_file['sentiment'] = rfc1

In [35]:
# submission_file['sentiment'] = y_pred

In [36]:
submission_file

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,1
...,...,...
24995,2155_10,1
24996,59_10,1
24997,2531_1,1
24998,7772_8,1


In [37]:
submission_file.to_csv('Using ensemble model.csv',index=False)

Using xgboost

In [38]:
import xgboost as xgb

In [39]:
xg_clf = xgb.XGBClassifier(objective ='binary:logistic',learning_rate=0.2,n_estimators=1000,max_depth=20)


In [40]:
xg_clf.fit(X_train,y_train)


In [ ]:
y_val_pred = xg_clf.predict(X_val)

In [ ]:
print(roc_auc_score(y_val_pred,y_val))

0.8165445555738109


In [ ]:
xgb1 = xg_clf.predict(X_test)
xgb2 = xg_clf.predict_proba(X_test)[:,1]
     

In [ ]:
xgb1

array([1, 0, 1, ..., 1, 1, 1])

In [ ]:
xgb2

array([0.9999751 , 0.00273723, 0.9530042 , ..., 0.8895306 , 0.99703336,
       0.9685051 ], dtype=float32)

In [ ]:
# putting result in submission file

submission_file = pd.read_csv('sampleSubmission.csv')
submission_file.head()

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0


In [ ]:
submission_file['sentiment'] = y_pred
submission_file.to_csv('Using xgboost.csv',index=False)

Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)
y_val_pred = lr.predict(X_val)

print(roc_auc_score(y_val_pred,y_val))

lr1 = lr.predict(X_test)
lr2 = lr.predict_proba(X_test)[:,1]

0.8644982811696159


In [ ]:
lr1

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
lr2

array([0.84992342, 0.20572088, 0.66644027, ..., 0.57341154, 0.82814881,
       0.58303576])

In [ ]:
# putting result in submission file

submission_file = pd.read_csv('sampleSubmission.csv')
submission_file['sentiment'] = lr1
submission_file.to_csv('Using Logistic Regression.csv',index=False)

Model Ensembling

In [ ]:
# For random forest 
print(rfc2)     

[0.93 0.17 0.62 ... 0.61 0.75 0.55]


In [ ]:
# For xgboost
print(xgb2)
     

[0.9999751  0.00273723 0.9530042  ... 0.8895306  0.99703336 0.9685051 ]


In [ ]:
# For logistic
print(lr2)

[0.84992342 0.20572088 0.66644027 ... 0.57341154 0.82814881 0.58303576]


In [ ]:
a = rfc2+xgb2+lr2
a = a/3
print(a)

[0.92663283 0.1261527  0.74648148 ... 0.69098071 0.85839406 0.70051361]


In [ ]:
submission_file = pd.read_csv('sampleSubmission.csv')
submission_file['sentiment'] = a
submission_file.to_csv('Model Mixing.csv',index=False)

Using Naive Base

In [ ]:
# Using naive base

from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix 
  
clf = GaussianNB()
clf.fit(X_train.toarray(), y_train) 
  
# predicting test set results 
y_val_pred = clf.predict(X_val.toarray()) 
  
# making the confusion matrix 
cm = confusion_matrix(y_val, y_val_pred) 
print(cm)

print(roc_auc_score(y_val_pred,y_val))

c1 = clf.predict(X_test.toarray())
c2 = clf.predict_proba(X_test.toarray())[:,1]

[[811 423]
 [437 829]]
0.6559901593348079


In [ ]:
submission_file = pd.read_csv('sampleSubmission.csv')
submission_file['sentiment'] = c1
submission_file.to_csv('NB.csv',index=False)